In [2]:
from google.colab import drive
import tarfile, os

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/aclImdb_v1.tar.gz'
extract_path = '/content/aclImdb'

with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(extract_path)

print(os.listdir(extract_path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-1476597930.py:10: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(extract_path)


['aclImdb']


In [3]:
import pandas as pd

def load_imdb(folder):
    texts, labels = [], []
    for label in ['pos', 'neg']:
        path = os.path.join(folder, label)
        for f in os.listdir(path):
            if f.endswith('.txt'):
                with open(os.path.join(path, f), encoding='utf-8') as file:
                    texts.append(file.read())
                labels.append(1 if label == 'pos' else 0)
    return pd.DataFrame({'review': texts, 'label': labels})

train_path = '/content/aclImdb/aclImdb/train'
test_path  = '/content/aclImdb/aclImdb/test'

df_train = load_imdb(train_path)
df_test  = load_imdb(test_path)

print(df_train.shape, df_test.shape)


(25000, 2) (25000, 2)


In [4]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train['review'])

X_train = tokenizer.texts_to_sequences(df_train['review'])
X_test  = tokenizer.texts_to_sequences(df_test['review'])

X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test  = pad_sequences(X_test,  maxlen=max_len, padding='post', truncating='post')

y_train = df_train['label'].values
y_test  = df_test['label'].values


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

embedding_dim = 128

model = Sequential([
    Embedding(max_words, embedding_dim),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=3,
    batch_size=256
)


Epoch 1/3
79/79 ━━━━━━━━━━━━━━━━━━━━ 170s 2s/step - accuracy: 0.6345 - loss: 0.6504 - val_accuracy: 0.6646 - val_loss: 0.5052
Epoch 2/3
79/79 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.6594 - loss: 0.6127 - val_accuracy: 0.2952 - val_loss: 0.8119
Epoch 3/3
79/79 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.6561 - loss: 0.6448 - val_accuracy: 0.2392 - val_loss: 0.8204


In [8]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Embedding(input_dim=max_words, output_dim=128),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.0003),  # 👈 very important
    metrics=['accuracy']
)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=1,
    restore_best_weights=True
)


In [10]:
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=256,
    callbacks=[early_stop]
)


Epoch 1/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.6147 - loss: 0.6715 - val_accuracy: 0.1650 - val_loss: 0.8739
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.7390 - loss: 0.5279 - val_accuracy: 0.7350 - val_loss: 0.6767
Epoch 3/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.8426 - loss: 0.4013 - val_accuracy: 0.8282 - val_loss: 0.4577
Epoch 4/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.8795 - loss: 0.3200 - val_accuracy: 0.8256 - val_loss: 0.5556


In [11]:
import pickle

model.save("imdb_sentiment_model.h5")

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("Final model saved")


Final model saved


In [12]:
def predict_review(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=200, padding='post', truncating='post')
    pred = model.predict(padded)[0][0]
    return pred, ("Positive" if pred > 0.5 else "Negative")


print(predict_review("I loved this movie, it was fantastic"))
print(predict_review("Worst movie ever, total waste of time"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
(np.float32(0.29278412), 'Negative')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
(np.float32(0.15703073), 'Negative')
